# Creating a neural network in PyTorch
[A PyTorch tutorial – deep learning in Python](https://adventuresinmachinelearning.com/pytorch-tutorial-deep-learning/)

Here we will create a simple 4-layer  fully connected neural network (including an “input layer” and two hidden layers) to classify the hand-written digits of the MNIST dataset. The architecture we’ll use can be seen in the figure below:

![Model FullyCNN](assets/CNTK-Dense-example-architecture.jpg)

The input layer consists of 28 x 28 (=784) greyscale pixels which constitute the input data of the MNIST data set. This input is then passed through two fully connected hidden layers, each with 200 nodes, with the nodes utilizing a [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) activation function. Finally, we have an output layer with ten nodes corresponding to the 10 possible classes of hand-written digits (i.e. 0 to 9). We will use a [softmax](https://en.wikipedia.org/wiki/Softmax_function) output layer to perform this classification.

## The neural network class

In order to create a neural network in PyTorch, you need to use the included class nn.Module. To use this base class, we also need to use Python class inheritance – this basically allows us to use all of the functionality of the nn.Module base class, but still have overwriting capabilities of the base class for the model construction / forward pass through the network. Some actual code will help explain.

```python
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
```

In the class definition, you can see the inheritance of the base class nn.Module. Then, in the first line of the class initialization (def __init__(self):) we have the required Python super() function, which creates an instance of the base nn.Module class. The following three lines is where we create our fully connected layers as per the architecture diagram. A fully connected neural network layer is represented by the nn.Linear object, with the first argument in the definition being the number of nodes in layer l and the next argument being the number of nodes in layer l+1. As you can observer, the first layer takes the 28 x 28 input pixels and connects to the first 200 node hidden layer. Then we have another 200 to 200 hidden layer, and finally a connection between the last hidden layer and the output layer (with 10 nodes).

Now we’ve setup the “skeleton” of our network architecture, we have to define how data flows through out network. We do this by defining a forward() method in our class – this method overwrites a dummy method in the base class, and needs to be defined for each network:

```python
def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x)
```

For the forward() method, we supply the input data x as the primary argument. We feed this into our first fully connected layer (self.fc1(x)) and then apply a ReLU activation to the nodes in this layer using F.relu(). Because of the hierarchical nature of this network, we replace x at each stage, feeding it into the next layer. We do this through our three fully connected layers, except for the last one – instead of a ReLU activation we return a log softmax “activation”. This, combined with the negative log likelihood loss function which will be defined later, gives us a multi-class cross entropy based loss function which we will use to train the network.

The next step is to create an instance of this network architecture:

```python
net = Net()
print(net)
```

When we print the instance of the class Net, we get the following output:

```text
Net (
(fc1): Linear (784 -> 200)
(fc2): Linear (200 -> 200)
(fc3): Linear (200 -> 10)
)
```

This is pretty handy as it confirms the structure of our network for us.

## Training the network

Next we have to setup an optimizer and a loss criterion:

```python
# create a stochastic gradient descent optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
# create a loss function
criterion = nn.NLLLoss()
```

In the first line, we create a stochastic gradient descent optimizer, and we specify the learning rate (which I’ve passed to this function as 0.01) and a momentum of 0.9. The other ingredient we need to supply to our optimizer is all the parameters of our network – thankfully PyTorch make supplying these parameters easy by the .parameters() method of the base nn.Module class that we inherit from in the Net class.

Next, we set our loss criterion to be the negative log likelihood loss – this combined with our log softmax output from the neural network gives us an equivalent cross entropy loss for our 10 classification classes.

Now it’s time to train the network. During training, I will be extracting data from a data loader object which is included in the PyTorch utilities module. This data loader will supply batches of input and target data which we’ll supply to our network and loss function respectively. Here’s the full training code:

```python
# run the main training loop
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        data = data.view(-1, 28*28)
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
```

The outer training loop is the number of epochs, whereas the inner training loop runs through the entire training set in batch sizes which are specified in the code as batch_size. On the next line, we convert data and target into PyTorch variables. The MNIST input data-set which is supplied in the torchvision package (which you’ll need to install using pip if you run the code for this tutorial) has the size (batch_size, 1, 28, 28) when extracted from the data loader – this 4D tensor is more suited to convolutional neural network architecture, and not so much our fully connected network. Therefore we need to flatten out the (1, 28, 28) data to a single dimension of 28 x 28 =  784 input nodes.


This lines: 

```python
data = data.view(-1, 28*28)
optimizer.zero_grad()
```


The .view() function operates on PyTorch variables to reshape them. If we want to be agnostic about the size of a given dimension, we can use the “-1” notation in the size definition. So by using data.view(-1, 28*28) we say that the second dimension must be equal to 28 x 28, but the first dimension should be calculated from the size of the original data variable. In practice, this means that data will now be of size (batch_size, 784). We can pass a batch of input data like this into our network and the magic of PyTorch will do all the hard work by efficiently performing the required operations on the tensors.

On the next line, we run optimizer.zero_grad() – this zeroes / resets all the gradients in the model, so that it is ready to go for the next back propagation pass. In other libraries this is performed implicitly, but in PyTorch you have to remember to do it explicitly. 

Let’s single out the next two lines:

```python
net_out = net(data)
loss = criterion(net_out, target)
```

The first line is where we pass the input data batch into the model – this will actually call the forward() method in our Net class. After this line is run, the variable net_out will now hold the log softmax output of our neural network for the given data batch. That’s one of the great things about PyTorch, you can activate whatever normal Python debugger you usually use and instantly get a gauge of what is happening in your network. This is opposed to other deep learning libraries such as TensorFlow and Keras which require elaborate debugging sessions to be setup before you can check out what your network is actually producing.

The second line is where we get the negative log likelihood loss between the output of our network and our target batch data.

```python
loss.backward()
optimizer.step()
```

The first line here runs a back-propagation operation from the loss Variable backwards through the network. If you compare this with our review of the .backward() operation that we undertook earlier in this PyTorch tutorial, you’ll notice that we aren’t supplying the .backward() operation with an argument. Scalar variables, when we call .backward() on them, don’t require arguments – only tensors require a matching sized tensor argument to be passed to the .backward() operation.

The next line is where we tell PyTorch to execute a gradient descent step based on the gradients calculated during the .backward() operation.

Finally, we print out some results every time we reach a certain number of iterations:

```python
if batch_idx % log_interval == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
```

This print function shows our progress through the epochs and also gives the network loss at that point in the training. Note how you access the loss – you access the Variable .data property, which in this case will be a single valued array. We access the scalar loss by executing loss.data[0].

Running this training loop you’ll get an output that looks something like this:

```text
rain Epoch: 9 [52000/60000 (87%)] Loss: 0.015086
Train Epoch: 9 [52000/60000 (87%)] Loss: 0.015086
Train Epoch: 9 [54000/60000 (90%)] Loss: 0.030631
Train Epoch: 9 [56000/60000 (93%)] Loss: 0.052631
Train Epoch: 9 [58000/60000 (97%)] Loss: 0.052678
```

After 10 epochs, you should get a loss value down around the <0.05 magnitude.

## Testing the network

To test the trained network on our test MNIST data set, we can run the following code:

```python
# run a test loop
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    data = data.view(-1, 28 * 28)
    net_out = net(data)
    # sum up batch loss
    test_loss += criterion(net_out, target).item()
    pred = net_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
```

This loop is the same as the previous training loop up until the test_loss line – here we extract the network loss using the .data[0] property as before, but all in the same line. Next, we have the pred line, where the data.max(1) method is used – this .max() method can return the index of the maximum value in a certain dimension of a tensor. Now, the output of our neural network will be of size (batch_size, 10), where each value of the 10-length second dimension is a log probability which the network assigns to each output class (i.e. it is the log probability of whether the given image is a digit between 0 and 9). So for each input sample/row in the batch, net_out.data will look something like this:

```text
[-1.3106e+01, -1.6731e+01, -1.1728e+01, -1.1995e+01, -1.5886e+01, -1.7700e+01, -2.4950e+01, -5.9817e-04, -1.3334e+01, -7.4527e+00]
```

The value with the highest log probability is the digit that the network considers to be the most probable given the input image – this is the best prediction of the class from the network. In the example of net_out.data above, it is the value -5.9817e-04 which is maximum, which corresponds to the digit “7”. So for this sample, the predicted digit is “7”. The .max(1) function will determine this maximum value in the second dimension (if we wanted the maximum in the first dimension, we’d supply an argument of 0) and returns both the maximum value that it has found, and the index that this maximum value was found at. It therefore has a size of (batch_size, 2) – in this case we are interested in the index where the maximum value is found at, therefore we access these values by calling .max(1)[1].

Now we have the prediction of the neural network for each sample in the batch determined, we can compare this with the actual target class from our training data, and count how many times in the batch the neural network got it right. We can use the PyTorch .eq() function to do this, which compares the values in two tensors and if they match, returns a 1. If they don’t match, it returns a 0:

```python
correct += pred.eq(target.data).sum()
```

By summing the output of the .eq() function, we get a count of the number of times the neural network has produced a correct output, and we take an accumulating sum of these correct predictions so that we can determine the overall accuracy of the network on our test data set. Finally, after running through the test data in batches, we print out the averaged loss and accuracy:

```python
test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
```

After training the network for 10 epochs, we get the following output from the above code on the test data:

```text
Test set: Average loss: 0.0003, Accuracy: 9783/10000 (98%)
```

A 98% accuracy – not bad!

So there you have it 

In [14]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


def simple_gradient():
    # print the gradient of 2x^2 + 5x
    x = Variable(torch.ones(2, 2) * 2, requires_grad=True)
    z = 2 * (x * x) + 5 * x
    # run the backpropagation
    z.backward(torch.ones(2, 2))
    print(x.grad)


def create_nn(batch_size=200, learning_rate=0.01, epochs=3,
              log_interval=10):

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(28 * 28, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 10)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)

    net = Net()
    print(net)

    # create a stochastic gradient descent optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # create a loss function
    criterion = nn.NLLLoss()

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))

    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 28 * 28)
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).item()
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.331256


/Users/a.calderon.machuca/.local/share/virtualenvs/kaka-sPsbq-e8/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.208768
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.941722
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.358670
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.792633
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.635027
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.563788
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.441449
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.435862
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.471460
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.382576
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.230259
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.335611
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.308573
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.337263
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.300235
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.275453
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.274764
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.259939
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.306546
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.401564
Train 

/Users/a.calderon.machuca/.local/share/virtualenvs/kaka-sPsbq-e8/lib/python3.6/site-packages/ipykernel_launcher.py:76: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0006, Accuracy: 9622/10000 (96%)

